In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import os
from scipy.sparse import csr_matrix, hstack

Using TensorFlow backend.


In [2]:
#查看文件列表
import os
print(os.listdir("./input"))
datadir = './input'
df_train = pd.read_csv(os.path.join(datadir,'age_train.csv'),header=None, names=['uId','age_group'],dtype = {'uId':np.str},index_col = 'uId')
df_train.head()
df_test = pd.read_csv(os.path.join(datadir,'age_test.csv'),names=['uId'],dtype = {'uId':np.str,},index_col = 'uId')
df_test.head()

['40dims.npy', 'age_test.csv', 'age_train.csv', 'app_info.csv', 'app_info_duplicates.csv', 'df_all.csv', 'merge_age_basic_info_app_actived_test.csv', 'merge_age_basic_info_app_actived_train.csv', 'user_app_actived.csv', 'user_app_actived_label.csv', 'user_app_usage.csv', 'user_app_usage20190301.csv', 'user_app_usage_aver_time.csv', 'user_basic_info.csv', 'user_behavior_info.csv']


""
uId
1000002
1000003
1000004
1000005
1000006


# 读取训练数据和测试数据文件

In [3]:
df_train = pd.read_csv(os.path.join(datadir,'merge_age_basic_info_app_actived_train.csv'),usecols=['uId','gender','prodName','color','carrier','age_group',
                                                                                                   'city','ct', 'os'],dtype = {'uId':np.str,'os':np.str},index_col = 'uId')
df_train.head()

,age_group,gender,city,prodName,color,ct,carrier,os
uId,,,,,,,,
1000001,4,0,c00145,p00169,皓月银,4g,China_Mobile,8.0
1000011,3,0,c00126,p0023,幻夜黑,4g#wifi,China_Mobile,8.1
1000015,5,1,c00306,p00169,皓月银,wifi,China_Telecom,8.0
1000019,3,0,c00150,p0049,蓝色,NaN,China_Unicom,8.1
1000023,2,1,c0037,p0047,银色,wifi,China_Telecom,7.0


In [4]:
df_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age_group,2.01e+06,NaN,NaN,NaN,3.51244,1.25428,1,3,3,4,6
gender,2.01e+06,NaN,NaN,NaN,0.212242,0.408895,0,0,0,0,1
city,2001683,361,c0043,149614,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prodName,2010000,219,p00108,123077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
color,2010000,136,幻夜黑,209743,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ct,1896557,7,4g#wifi,1045062,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carrier,2010000,4,China_Mobile,897322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
os,2009016,16,8.0,892890,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_test = pd.read_csv(os.path.join(datadir,'merge_age_basic_info_app_actived_test.csv'),usecols=['uId','gender','prodName','color','carrier',
                                                                                                 'city','ct', 'os'],dtype = {'uId':np.str,'os':np.str},index_col = 'uId')
df_test.head()

,gender,city,prodName,color,ct,carrier,os
uId,,,,,,,
1000002,1,c0043,p0022,渐变黑,wifi,China_Mobile,9.0
1000003,0,c00302,p00103,翡冷翠,4g#wifi,China_Mobile,9.0
1000004,1,c00281,p0013,幻海蓝,4g#wifi,China_Mobile,8.0
1000005,0,c00199,p001,幻夜黑,4g#wifi,China_Mobile,9.0
1000006,0,c00359,p0045,灰色,NaN,China_Mobile,6.0


In [6]:
df_test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
gender,502500,NaN,NaN,NaN,0.212633,0.40917,0,0,0,0,1
city,500298,356,c0043,36941,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prodName,502500,193,p00108,30322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
color,502500,128,幻夜黑,53225,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ct,474390,7,4g#wifi,261679,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carrier,502500,4,China_Mobile,223896,NaN,NaN,NaN,NaN,NaN,NaN,NaN
os,502256,13,8.0,222452,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 将训练集与测试集拼接，目的是进行NaN填充，和特征构造

In [7]:
df_all = pd.concat([df_train,df_test], sort=False)
df_all

,age_group,gender,city,prodName,color,ct,carrier,os
uId,,,,,,,,
1000001,4.0,0,c00145,p00169,皓月银,4g,China_Mobile,8.0
1000011,3.0,0,c00126,p0023,幻夜黑,4g#wifi,China_Mobile,8.1
1000015,5.0,1,c00306,p00169,皓月银,wifi,China_Telecom,8.0
1000019,3.0,0,c00150,p0049,蓝色,NaN,China_Unicom,8.1
1000023,2.0,1,c0037,p0047,银色,wifi,China_Telecom,7.0
1000025,4.0,0,c006,p00171,琥珀金,4g#wifi,China_Telecom,8.0
1000029,4.0,0,c0093,p00156,极光色,4g#wifi,China_Unicom,9.0
1000035,2.0,0,c00316,p00156,宝石蓝,wifi,China_Telecom,9.0
1000037,4.0,0,c00324,p00107,香槟金,wifi,China_Unicom,8.0


# 填充缺失值,众数填充。需要改进，在整个数据集找众数

In [8]:
for name in ['gender','prodName','color','carrier',  'city','ct', 'os']:
    mode_train = df_all[name].mode()
    mode_test = df_all[name].mode()
    df_all[name].fillna(mode_train[0], inplace=True)
    df_train[name].fillna(mode_train[0], inplace=True)
    df_test[name].fillna(mode_test[0], inplace=True)

In [9]:
print(df_train['prodName'].mode())

0    p00108
dtype: object


In [10]:
df_train.head()

,age_group,gender,city,prodName,color,ct,carrier,os
uId,,,,,,,,
1000001,4,0,c00145,p00169,皓月银,4g,China_Mobile,8.0
1000011,3,0,c00126,p0023,幻夜黑,4g#wifi,China_Mobile,8.1
1000015,5,1,c00306,p00169,皓月银,wifi,China_Telecom,8.0
1000019,3,0,c00150,p0049,蓝色,4g#wifi,China_Unicom,8.1
1000023,2,1,c0037,p0047,银色,wifi,China_Telecom,7.0


In [11]:
df_train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age_group,2.01e+06,NaN,NaN,NaN,3.51244,1.25428,1,3,3,4,6
gender,2.01e+06,NaN,NaN,NaN,0.212242,0.408895,0,0,0,0,1
city,2010000,361,c0043,157931,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prodName,2010000,219,p00108,123077,NaN,NaN,NaN,NaN,NaN,NaN,NaN
color,2010000,136,幻夜黑,209743,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ct,2010000,7,4g#wifi,1158505,NaN,NaN,NaN,NaN,NaN,NaN,NaN
carrier,2010000,4,China_Mobile,897322,NaN,NaN,NaN,NaN,NaN,NaN,NaN
os,2010000,16,8.0,893874,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 构造特征

In [12]:
df_train['trainrow'] = np.arange(df_train.shape[0])
df_test['testrow'] = np.arange(df_test.shape[0])

In [13]:
df_train.head()

,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,c00145,p00169,皓月银,4g,China_Mobile,8.0,0
1000011,3,0,c00126,p0023,幻夜黑,4g#wifi,China_Mobile,8.1,1
1000015,5,1,c00306,p00169,皓月银,wifi,China_Telecom,8.0,2
1000019,3,0,c00150,p0049,蓝色,4g#wifi,China_Unicom,8.1,3
1000023,2,1,c0037,p0047,银色,wifi,China_Telecom,7.0,4


In [14]:
df_test.head()

,gender,city,prodName,color,ct,carrier,os,testrow
uId,,,,,,,,
1000002,1,c0043,p0022,渐变黑,wifi,China_Mobile,9.0,0
1000003,0,c00302,p00103,翡冷翠,4g#wifi,China_Mobile,9.0,1
1000004,1,c00281,p0013,幻海蓝,4g#wifi,China_Mobile,8.0,2
1000005,0,c00199,p001,幻夜黑,4g#wifi,China_Mobile,9.0,3
1000006,0,c00359,p0045,灰色,4g#wifi,China_Mobile,6.0,4


dender

In [15]:
#'gender','prodName','color','carrier',  'city','ct', 'os'

brandencoder = LabelEncoder().fit(df_all.gender)
df_all['gender_T'] = brandencoder.transform(df_all.gender)

df_train['gender'] = df_all['gender_T']
df_test['gender'] = df_all['gender_T']

Xtr_gender = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.gender)))
Xte_gender = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.gender)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_gender.shape, Xte_gender.shape))
df_train.head()

Brand features: train shape (2010000, 2), test shape (502500, 2)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,c00145,p00169,皓月银,4g,China_Mobile,8.0,0
1000011,3,0,c00126,p0023,幻夜黑,4g#wifi,China_Mobile,8.1,1
1000015,5,1,c00306,p00169,皓月银,wifi,China_Telecom,8.0,2
1000019,3,0,c00150,p0049,蓝色,4g#wifi,China_Unicom,8.1,3
1000023,2,1,c0037,p0047,银色,wifi,China_Telecom,7.0,4


prodName

In [16]:
#'gender','prodName','color','carrier',  'city','ct', 'os'

brandencoder = LabelEncoder().fit(df_all.prodName)
df_all['prodName_T'] = brandencoder.transform(df_all.prodName)

df_train['prodName'] = df_all['prodName_T']
df_test['prodName'] = df_all['prodName_T']

Xtr_prodName = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.prodName)))
Xte_prodName = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.prodName)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_prodName.shape, Xte_prodName.shape))
df_train.head()

Brand features: train shape (2010000, 227), test shape (502500, 227)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,c00145,77,皓月银,4g,China_Mobile,8.0,0
1000011,3,0,c00126,143,幻夜黑,4g#wifi,China_Mobile,8.1,1
1000015,5,1,c00306,77,皓月银,wifi,China_Telecom,8.0,2
1000019,3,0,c00150,171,蓝色,4g#wifi,China_Unicom,8.1,3
1000023,2,1,c0037,169,银色,wifi,China_Telecom,7.0,4


color

In [17]:
#'gender','prodName','color','carrier',  'city','ct', 'os'
brandencoder = LabelEncoder().fit(df_all.color)
df_all['color_T'] = brandencoder.transform(df_all.color)

df_train['color'] = df_all['color_T']
df_test['color'] = df_all['color_T']

Xtr_color = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.color)))
Xte_color = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.color)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_color.shape, Xte_color.shape))
df_train.head()

Brand features: train shape (2010000, 136), test shape (502500, 136)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,c00145,77,79,4g,China_Mobile,8.0,0
1000011,3,0,c00126,143,15,4g#wifi,China_Mobile,8.1,1
1000015,5,1,c00306,77,79,wifi,China_Telecom,8.0,2
1000019,3,0,c00150,171,99,4g#wifi,China_Unicom,8.1,3
1000023,2,1,c0037,169,118,wifi,China_Telecom,7.0,4


carrier

In [18]:
#'gender','prodName','color','carrier',  'city','ct', 'os'
brandencoder = LabelEncoder().fit(df_all.carrier)
df_all['carrier_T'] = brandencoder.transform(df_all.carrier)

df_train['carrier'] = df_all['carrier_T']
df_test['carrier'] = df_all['carrier_T']

Xtr_carrier = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.carrier)))
Xte_carrier = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.carrier)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_carrier.shape, Xte_carrier.shape))
df_train.head()

Brand features: train shape (2010000, 4), test shape (502500, 4)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,c00145,77,79,4g,0,8.0,0
1000011,3,0,c00126,143,15,4g#wifi,0,8.1,1
1000015,5,1,c00306,77,79,wifi,1,8.0,2
1000019,3,0,c00150,171,99,4g#wifi,2,8.1,3
1000023,2,1,c0037,169,118,wifi,1,7.0,4


city

In [19]:
#'gender','prodName','color','carrier',  'city','ct', 'os'
brandencoder = LabelEncoder().fit(df_all.city)
df_all['city_T'] = brandencoder.transform(df_all.city)

df_train['city'] = df_all['city_T']
df_test['city'] = df_all['city_T']

Xtr_city = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.city)))
Xte_city = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.city)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_city.shape, Xte_city.shape))
df_train.head()

Brand features: train shape (2010000, 363), test shape (502500, 363)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,50,77,79,4g,0,8.0,0
1000011,3,0,29,143,15,4g#wifi,0,8.1,1
1000015,5,1,229,77,79,wifi,1,8.0,2
1000019,3,0,56,171,99,4g#wifi,2,8.1,3
1000023,2,1,294,169,118,wifi,1,7.0,4


ct

In [20]:
#'gender','prodName','color','carrier',  'city','ct', 'os'
brandencoder = LabelEncoder().fit(df_all.ct)
df_all['ct_T'] = brandencoder.transform(df_all.ct)

df_train['ct'] = df_all['ct_T']
df_test['ct'] = df_all['ct_T']

Xtr_ct = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.ct)))
Xte_ct = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.ct)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_ct.shape, Xte_ct.shape))
df_train.head()

Brand features: train shape (2010000, 7), test shape (502500, 7)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,50,77,79,4,0,8.0,0
1000011,3,0,29,143,15,5,0,8.1,1
1000015,5,1,229,77,79,6,1,8.0,2
1000019,3,0,56,171,99,5,2,8.1,3
1000023,2,1,294,169,118,6,1,7.0,4


os

In [21]:
#'gender','prodName','color','carrier',  'city','ct', 'os'
brandencoder = LabelEncoder().fit(df_all.os)
df_all['os_T'] = brandencoder.transform(df_all.os)

df_train['os'] = df_all['os_T']
df_test['os'] = df_all['os_T']

Xtr_os = csr_matrix((np.ones(df_train.shape[0]), 
                       (df_train.trainrow, df_train.os)))
Xte_os = csr_matrix((np.ones(df_test.shape[0]), 
                       (df_test.testrow, df_test.os)))
print('Brand features: train shape {}, test shape {}'.format(Xtr_os.shape, Xte_os.shape))
df_train.head()

Brand features: train shape (2010000, 16), test shape (502500, 16)


,age_group,gender,city,prodName,color,ct,carrier,os,trainrow
uId,,,,,,,,,
1000001,4,0,50,77,79,4,0,13,0
1000011,3,0,29,143,15,5,0,14,1
1000015,5,1,229,77,79,6,1,13,2
1000019,3,0,56,171,99,5,2,14,3
1000023,2,1,294,169,118,6,1,11,4


# 拼接数据

In [22]:
#'gender','prodName','color','carrier',  'city','ct', 'os'
Xtrain = hstack((Xtr_gender, Xtr_prodName, Xtr_color, Xtr_carrier, Xtr_city, Xtr_ct, Xtr_os,Xtr_app9401), format='csr')
Xtest =  hstack((Xte_gender, Xte_prodName, Xte_color, Xte_carrier, Xte_city, Xte_ct, Xte_os,Xte_app9401), format='csr')
print('All features: train shape {}, test shape {}'.format(Xtrain.shape, Xtest.shape))

All features: train shape (2010000, 755), test shape (502500, 755)


# sklearn交叉验证

In [23]:
targetencoder = LabelEncoder().fit(df_train.age_group)
y = targetencoder.transform(df_train.age_group)
nclasses = len(targetencoder.classes_)

In [25]:
def score(clf, random_state = 0):
    kf = StratifiedKFold(n_splits=5,shuffle=True, random_state=random_state)
    pred = np.zeros((y.shape[0],nclasses))
    for itrain, itest in kf.split(Xtrain,y):
        Xtr, Xte = Xtrain[itrain, :], Xtrain[itest, :]
        ytr, yte = y[itrain], y[itest]
        clf.fit(Xtr, ytr)
        pred[itest,:] = clf.predict_proba(Xte)
        # Downsize to one fold only for kernels
        return log_loss(yte, pred[itest, :])
        print("{:.5f}".format(log_loss(yte, pred[itest,:])), end=' ')
    print('')
    return log_loss(y, pred)

In [27]:
clf = LogisticRegression(C=0.1, multi_class='multinomial',solver='lbfgs')

In [28]:
clf = LogisticRegression(C=0.1, multi_class='multinomial',solver='lbfgs')
clf.fit(Xtrain, y)
pred = pd.DataFrame(clf.predict(Xtest), index = df_test.index)
pred.head()

C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


,0
uId,
1000002,2
1000003,2
1000004,1
1000005,1
1000006,3


In [29]:
pred.rename(columns={0: 'label'}, inplace=True)
pred.index.names = ['id']
pred['label'] = targetencoder.inverse_transform(pred['label'])

In [30]:
pred.head()

,label
id,
1000002,3
1000003,3
1000004,2
1000005,2
1000006,4


In [31]:
pred.to_csv('submission.csv',index=True)